In [ ]:
import sys
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from skimage.feature import hog
from skimage.transform import resize
from skimage.color import rgb2gray
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import joblib
import os
import matplotlib.pyplot as plt

CIFAR10_CLASSES = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
MODEL_FILENAME = "image_classifier_model.pkl"
IMAGE_SIZE = (64, 64)

def extract_features(image):
    image_gray = rgb2gray(image)
    image_resized = resize(image_gray, IMAGE_SIZE)

    features = hog(image_resized, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False, channel_axis=None)
    return features

def train_and_save_model():
    print("Baixando e carregando o dataset CIFAR-10. Isso pode levar alguns minutos...")
    cifar_10 = fetch_openml('CIFAR_10', version=1, as_frame=False)

    X = cifar_10.data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    y = np.array([int(label) for label in cifar_10.target])

    print("Dataset carregado. Extraindo características das imagens de treino...")

    X_features = np.array([extract_features(img) for img in X])

    X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)

    print("Treinando o classificador KNN...")

    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(X_train, y_train)

    print("Avaliando o modelo...")
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Acurácia do modelo nos dados de teste: {accuracy:.4f}")

    print(f"Salvando o modelo treinado em '{MODEL_FILENAME}'...")
    joblib.dump(model, MODEL_FILENAME)

    return model

def load_model():
    if os.path.exists(MODEL_FILENAME):
        return joblib.load(MODEL_FILENAME)
    else:
        print("Modelo não encontrado. Iniciando o treinamento de um novo modelo...")
        return train_and_save_model()


def classify_image_from_url(model, url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        img_bytes = BytesIO(response.content)
        image = Image.open(img_bytes).convert('RGB')
        image_np = np.array(image)

        plt.imshow(image)
        plt.axis('off')
        plt.title("Imagem Recebida")
        plt.show()


        features = extract_features(image_np)

        prediction_index = model.predict([features])[0]
        predicted_class = CIFAR10_CLASSES[prediction_index]

        return predicted_class

    except requests.exceptions.RequestException as e:
        print(f"Erro no download da imagem: {e}")
        return None
    except Exception as e:
        print(f"Erro classify_image_from_url: {e}")
        return None


image_url = "https://cdn.awsli.com.br/2500x2500/78/78037/produto/243383755/papo-50164-king-cobra-282f6lg1g1.jpg"

classifier = load_model()

print("\nModelo carregado com sucesso!")
print(f"Classes suportadas: {CIFAR10_CLASSES}")
print(f"Classificando a imagem da URL: {image_url}")

result = classify_image_from_url(classifier, image_url)

if result:
    print(f"\n>> A imagem foi classificada como: '{result}'")
else:
    print("\n>> Falha ao classificar a imagem. Verifique a URL ou tente novamente.")


Modelo não encontrado. Iniciando o treinamento de um novo modelo...
Baixando e carregando o dataset CIFAR-10. Isso pode levar alguns minutos...
